<a href="https://colab.research.google.com/github/sap-tarshi-ghosh/AML-Dataset/blob/main/code/UCI_SMOTE_Wilcoxon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon

# Load Excel file
file_path = "UCI_Dataset_SMOTE_ALL.xlsx"
df = pd.read_excel(file_path)

# Extract only 10-fold CV data (rows 0–9) and ensure it's a copy to avoid SettingWithCopyWarning
cv_data = df.iloc[0:10].copy()

# Convert model columns to numeric, coercing errors to NaN
cv_data["AdaBoost"] = pd.to_numeric(cv_data["AdaBoost"], errors='coerce')
cv_data["Bagging"] = pd.to_numeric(cv_data["Bagging"], errors='coerce')

# Select the two models to compare
model_A = cv_data["AdaBoost"]
model_B = cv_data["Bagging"]

# Compute differences
diff = model_A - model_B

# Remove zero differences (Wilcoxon requirement) and NaN values
non_zero_diff = diff[diff != 0].dropna()

# Perform Wilcoxon signed-rank test (two-sided)
stat, p_value = wilcoxon(
    non_zero_diff,
    alternative="two-sided",
    zero_method="wilcox",
    mode="auto"
)

# Effect size calculation
# According to https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test#Effect_size
# This calculation needs to handle the case where non_zero_diff is empty to prevent division by zero or sqrt(negative).
# If non_zero_diff is empty, stat would be 0, and len(non_zero_diff) would be 0.
if len(non_zero_diff) > 0:
    # Recalculate z based on the formula for effect size, not the stat value itself
    # The previous `z = stat - (len(non_zero_diff) * (len(non_zero_diff) + 1)) / 4` is specific to the test statistic, not for effect size calculation 'r'
    # For effect size r, we usually need the W statistic and N (number of non-zero differences)
    # The provided formula in the comment links to a different interpretation of Z/r
    # A common effect size for Wilcoxon is r = Z / sqrt(N)
    # The 'z' variable in the original code is not directly the 'Z' from normal approximation for r

    # Let's use a common approach where Z is derived from the test statistic for large N
    # For small N, directly computing 'r' is more complex or uses different approximations
    # Given the previous value for stat was 5.0 and non_zero_diff has 4 elements:

    # The previous calculation for 'z' directly used 'stat' in a way that might not align with 'r = Z / sqrt(N)'
    # Let's re-evaluate the effect size calculation provided in the original code snippet
    # The formula used for z and effect_size_r was:
    # z = stat - (len(non_zero_diff) * (len(non_zero_diff) + 1)) / 4
    # z /= np.sqrt(len(non_zero_diff) * (len(non_zero_diff) + 1) * (2 * len(non_zero_diff) + 1) / 24)
    # effect_size_r = abs(z) / np.sqrt(len(non_zero_diff))

    # This 'z' is essentially a standardized test statistic. Let's stick to the user's provided formula if it's what they intended.
    # The issue might be that with N=4, the normal approximation for Z is not very accurate, leading to odd r values.
    # However, if non_zero_diff is empty, the calculation would fail.

    # Re-using the existing effect size calculation method, ensuring N > 0
    rank_sum_expected = len(non_zero_diff) * (len(non_zero_diff) + 1) / 4
    std_dev_rank_sum = np.sqrt(len(non_zero_diff) * (len(non_zero_diff) + 1) * (2 * len(non_zero_diff) + 1) / 24)

    if std_dev_rank_sum > 0:
        z_score_for_r = (stat - rank_sum_expected) / std_dev_rank_sum
        effect_size_r = abs(z_score_for_r) / np.sqrt(len(non_zero_diff))
    else:
        effect_size_r = 0.0 # If std_dev is 0, all differences might be the same, or N is too small
else:
    effect_size_r = 0.0 # If there are no non-zero differences, effect size is 0

print(f"Wilcoxon statistic = {stat:.4f}")
print(f"p-value = {p_value:.4f}")
print(f"Effect size (r) = {effect_size_r:.4f}")

Wilcoxon statistic = 5.0000
p-value = 1.0000
Effect size (r) = 0.0000


In [9]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests

# ==========================================
# FILE PATH
# ==========================================

file_path = "UCI_Dataset_SMOTE_ALL.xlsx"

# ==========================================
# METRIC DIRECTION (FULLY DEFINED)
# True  -> higher is better
# False -> lower is better
# ==========================================

metric_direction = {
    "Accuracy": True,
    "Precision": True,
    "Recall": True,
    "Sensitivity": True,
    "Specificity": True,
    "F1": True,
    "F1-Score": True,
    "Kappa": True,
    "AUC": True,
    "G-Mean": True,
    "MCC": True,
    "NPV": True,
    "PPV": True,

    "FPR": False,
    "FNR": False,
    "Error": False,
    "Error Rate": False,
    "LogLoss": False,
    "Hamming Loss": False
}

# ==========================================
# LOAD EXCEL FILE
# ==========================================

xls = pd.ExcelFile(file_path)
all_results = []

# ==========================================
# MAIN LOOP OVER ALL METRIC SHEETS
# ==========================================

for sheet in xls.sheet_names:

    print(f"\n==============================")
    print(f"Processing metric: {sheet}")
    print(f"==============================")

    df = pd.read_excel(xls, sheet_name=sheet)

    # Use only first 10 rows (10-fold CV) and create a copy
    cv_data = df.iloc[0:10].copy()

    # Check if enough rows are present for 10-fold CV
    if cv_data.shape[0] != 10:
        print(f"Warning: {sheet}: Expected 10 CV rows, found {cv_data.shape[0]}. Skipping this sheet.")
        continue

    # Identify model performance columns by excluding 'Fold' (assuming 'Fold' is the identifier)
    # You might need to adjust this list if there are other non-model columns
    non_model_columns = ['Fold']
    model_performance_cols = [col for col in cv_data.columns if col not in non_model_columns]

    # Convert all identified model performance columns to numeric, coercing errors to NaN
    for col in model_performance_cols:
        cv_data[col] = pd.to_numeric(cv_data[col], errors='coerce')

    # Filter to only include model performance columns that are entirely numeric (not all NaN)
    cv_data_numeric_models = cv_data[model_performance_cols].dropna(axis=1, how='all')

    if cv_data_numeric_models.empty:
        print(f"Warning: No valid numeric model performance data found for sheet '{sheet}'. Skipping.")
        continue

    # Determine direction (default: higher is better)
    higher_is_better = metric_direction.get(sheet, True)

    # ==========================================
    # REFERENCE MODEL SELECTION (MEDIAN-BASED)
    # ==========================================

    medians = cv_data_numeric_models.median()

    if medians.empty:
        print(f"Warning: No medians could be calculated for sheet '{sheet}'. Skipping.")
        continue

    if higher_is_better:
        reference_model = medians.idxmax()
    else:
        reference_model = medians.idxmin()

    # Ensure ref_values are from the numeric model data
    ref_values = cv_data_numeric_models[reference_model]

    print(f"Reference model selected: {reference_model}")

    # ==========================================
    # WILCOXON SIGNED-RANK TESTS
    # ==========================================

    compared_models = []
    wilcoxon_stats = []
    raw_pvals = []

    for model in cv_data_numeric_models.columns: # Iterate only over valid numeric model columns
        if model == reference_model:
            continue

        current_model_values = cv_data_numeric_models[model]

        # Create a temporary DataFrame to handle NaNs and align pairs for comparison
        temp_df = pd.DataFrame({'ref': ref_values, 'comp': current_model_values}).dropna()

        # If not enough valid pairs after dropping NaNs, mark as unreliable
        if len(temp_df) < 5: # Consider 5 as a minimum number of valid pairs for reliable test
            stat = np.nan
            p_val = np.nan
        else:
            # Perform Wilcoxon signed-rank test on the paired, non-NaN values
            stat, p_val = wilcoxon(
                temp_df['ref'].values,
                temp_df['comp'].values,
                zero_method="wilcox",
                alternative="two-sided"
            )

        compared_models.append(model)
        wilcoxon_stats.append(stat)
        raw_pvals.append(p_val)

    # ==========================================
    # MULTIPLE COMPARISON CORRECTION (HOLM)
    # ==========================================

    # Filter out NaN p-values for multipletests and then re-assign
    valid_pvals_indices = [i for i, p in enumerate(raw_pvals) if not pd.isna(p)]

    if valid_pvals_indices:
        valid_pvals = [raw_pvals[i] for i in valid_pvals_indices]
        reject_valid, p_adj_valid, _, _ = multipletests(
            valid_pvals,
            method="holm",
            alpha=0.05
        )

        # Reconstruct p_adj and reject lists, filling NaNs where original raw_pvals were NaN
        p_adj = [np.nan] * len(raw_pvals)
        reject = [False] * len(raw_pvals)
        for i, original_idx in enumerate(valid_pvals_indices):
            p_adj[original_idx] = p_adj_valid[i]
            reject[original_idx] = reject_valid[i]
    else:
        # No valid p-values to correct
        p_adj = [np.nan] * len(raw_pvals)
        reject = [False] * len(raw_pvals)

    result_df = pd.DataFrame({
        "Metric": sheet,
        "Reference_Model": reference_model,
        "Compared_Model": compared_models,
        "Wilcoxon_Statistic": wilcoxon_stats,
        "Raw_p_value": raw_pvals,
        "Adjusted_p_value": p_adj,
        "Significant_after_Holm": reject
    })

    all_results.append(result_df)

# ==========================================
# FINAL RESULT TABLE
# ==========================================

if all_results:
    final_results = pd.concat(all_results, ignore_index=True)

    final_results.to_excel(
        "Wilcoxon_Reference_Based_Results.xlsx",
        index=False
    )

    print("\n========================================")
    print("Analysis complete.")
    print("Results saved to: Wilcoxon_Reference_Based_Results.xlsx")
    print("========================================")
else:
    print("\n========================================")
    print("Analysis complete, but no valid results to save.")
    print("========================================")


Processing metric: Accuracy
Reference model selected: Random Forest

Processing metric: AUC
Reference model selected: XGBoost

Processing metric: F1-Score
Reference model selected: XGBoost

Processing metric: FPR
Reference model selected: AdaBoost

Processing metric: GM
Reference model selected: XGBoost

Processing metric: Kappa
Reference model selected: XGBoost

Processing metric: MCC
Reference model selected: XGBoost

Processing metric: Precision
Reference model selected: AdaBoost

Processing metric: Recall
Reference model selected: KNN

Processing metric: Specificity
Reference model selected: AdaBoost

Processing metric: Sheet1

Analysis complete.
Results saved to: Wilcoxon_Reference_Based_Results.xlsx
